In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pip install neo4j
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt

In [48]:
url = "bolt://3.238.27.128:7687";
username = "neo4j"
password = "receptacle-plastics-scabs"

driver = GraphDatabase.driver(url, auth=(username, password))



In [49]:
floor_locations = ["Admission","Senserie","Bite Size Science","Cummings Room","3D Cinema","Earth", "Wind", "Jaeger Room"];
with driver.session() as session:
    # Perform database operations within the session
    for location in floor_locations:
      session.run("CREATE (n:room {name: '" + location +"'})")

In [50]:
with driver.session() as session:
  for i in range(len(floor_locations)-1):
    for j in range(len(floor_locations)):
      if floor_locations[i] != floor_locations[j]:
          query = (
                    "MATCH (source), (target) "
                    "WHERE source.name = $source_name AND target.name = $target_name "
                      "CREATE (source)-[:RELATIONSHIP {movement: '0'}]->(target)"
          )
          params = {
                    "source_name": floor_locations[i],
                    "target_name": floor_locations[j]
          }
          session.run(query,params)

In [51]:
def process_movement(path):
  for i in range(len(path)-1):
    with driver.session() as session:
      session.run("MATCH (source:room {name: $source_name})-[r:RELATIONSHIP]->(target:room {name: $target_name}) "
                "SET r.movement = toInteger(r.movement) + 1",
                source_name=path[i], target_name=path[i+1])


In [52]:
import random

def random_list(locations):
  to_random = floor_locations[1:]
  start = 1;
  end = len(locations)-1;
  random_number = random.randint(start, end);
  randomized_list = random.sample(to_random, random_number)
  return ["Admission"] + randomized_list + ["Admission"]



In [54]:
lanyard1 = ["Admission","Bite Size Science","3D Cinema","Wind","Admission"];
process_movement(lanyard1)

In [53]:
for i in range(100):
  lanyard = random_list(floor_locations)
  process_movement(lanyard)

In [47]:
with driver.session() as session:
     session.run("MATCH (n) DETACH DELETE n")